In [131]:
import pandas as pd
df = pd.read_csv("groceries.csv")

In [132]:
df

,Basket items
0,"citrus fruit,semi-finished bread,margarine,rea..."
1,"tropical fruit,yogurt,coffee"
2,whole milk
3,"pip fruit,yogurt,cream cheese,meat spreads"
4,"other vegetables,whole milk,condensed milk,lon..."
...,...
9830,"sausage,chicken,beef,hamburger meat,citrus fru..."
9831,cooking chocolate
9832,"chicken,citrus fruit,other vegetables,butter,y..."
9833,"semi-finished bread,bottled water,soda,bottled..."


In [133]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import association_rules, apriori
from mlxtend.preprocessing import TransactionEncoder
# get all the transcactions as a list
transcactions = list(df['Basket items'].apply(lambda x: sorted(x.split(','))))
# instantiate transcation encoder
encoder = TransactionEncoder().fit(transcactions)
onehot = encoder.transform(transcactions)
# convert one-hot encode data to DataFrame
onehot = pd.DataFrame(onehot, columns=encoder.columns_)
# compute frequent items using the Apriori algorithm - Get up to three items
frequent_itemsets = apriori(onehot, min_support = 0.001, max_len = 3, use_colnames=True)
# compute all association rules for frequent_itemsets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)


In [134]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(bottled water),0.008033,0.110524,0.001017,0.126582,1.145296,0.000129,1.018386
1,(bottled water),(Instant food products),0.110524,0.008033,0.001017,0.009200,1.145296,0.000129,1.001178
2,(butter),(Instant food products),0.055414,0.008033,0.001220,0.022018,2.741145,0.000775,1.014301
3,(Instant food products),(butter),0.008033,0.055414,0.001220,0.151899,2.741145,0.000775,1.113765
4,(citrus fruit),(Instant food products),0.082766,0.008033,0.001118,0.013514,1.682347,0.000454,1.005556
...,...,...,...,...,...,...,...,...,...
45715,"(white bread, whole milk)",(yogurt),0.017082,0.139502,0.004881,0.285714,2.048105,0.002498,1.204698
45716,"(yogurt, whole milk)",(white bread),0.056024,0.042095,0.004881,0.087114,2.069492,0.002522,1.049316
45717,(white bread),"(yogurt, whole milk)",0.042095,0.056024,0.004881,0.115942,2.069492,0.002522,1.067776
45718,(yogurt),"(white bread, whole milk)",0.139502,0.017082,0.004881,0.034985,2.048105,0.002498,1.018553


In [135]:
#rules['lhs items'] = rules['antecedents'].apply(lambda x:len(x) )
#rules[rules['lhs items']>1].sort_values('lift', ascending=False).head(10)

In [136]:
df = rules[['antecedents', 'consequents', 'lift']].copy()

In [137]:
df

,antecedents,consequents,lift
0,(Instant food products),(bottled water),1.145296
1,(bottled water),(Instant food products),1.145296
2,(butter),(Instant food products),2.741145
3,(Instant food products),(butter),2.741145
4,(citrus fruit),(Instant food products),1.682347
...,...,...,...
45715,"(white bread, whole milk)",(yogurt),2.048105
45716,"(yogurt, whole milk)",(white bread),2.069492
45717,(white bread),"(yogurt, whole milk)",2.069492
45718,(yogurt),"(white bread, whole milk)",2.048105


In [138]:
new_df = df[df.antecedents != ',']

In [139]:
new_df['antecedents'] = new_df['antecedents'].astype(str)

In [140]:
new_df['antecedents'] = new_df['antecedents'].str.split(',').str[0]

In [141]:
new_df

,antecedents,consequents,lift
0,frozenset({'Instant food products'}),(bottled water),1.145296
1,frozenset({'bottled water'}),(Instant food products),1.145296
2,frozenset({'butter'}),(Instant food products),2.741145
3,frozenset({'Instant food products'}),(butter),2.741145
4,frozenset({'citrus fruit'}),(Instant food products),1.682347
...,...,...,...
45715,frozenset({'white bread',(yogurt),2.048105
45716,frozenset({'yogurt',(white bread),2.069492
45717,frozenset({'white bread'}),"(yogurt, whole milk)",2.069492
45718,frozenset({'yogurt'}),"(white bread, whole milk)",2.048105


In [145]:
new_df[new_df.antecedents.apply(str).str.contains("pasta")].sort_values('lift', ascending=False).head(5)

,antecedents,consequents,lift
35001,frozenset({'pasta'}),"(whole milk, hamburger meat)",6.416123
3639,frozenset({'pasta'}),(hamburger meat),5.486920
16487,frozenset({'pasta'}),"(butter, root vegetables)",5.232496
3199,frozenset({'pasta'}),(frozen meals),4.287271
20001,frozenset({'pasta'}),"(chicken, other vegetables)",4.153294


In [153]:
s = new_df[new_df.antecedents.apply(str).str.contains("bread")].sort_values('lift', ascending=False).head(7)
s['consequents']

34592                   (processed cheese)
34593                                (ham)
34595              (ham, processed cheese)
29329    (domestic eggs, processed cheese)
29326                   (processed cheese)
43558                   (processed cheese)
42661        (processed cheese, pip fruit)
Name: consequents, dtype: object

In [152]:
q = df[df.antecedents.apply(str).str.contains("milk")].sort_values('lift', ascending=False).head(7)
q

,antecedents,consequents,lift
5440,"(Instant food products, whole milk)",(hamburger meat),15.038226
5442,"(whole milk, hamburger meat)",(Instant food products),12.878656
30000,"(sugar, whole milk)",(flour),10.881144
43636,"(sugar, whole milk)",(rice),10.632432
17183,(butter milk),"(dessert, pip fruit)",10.218182
29999,"(flour, whole milk)",(sugar),9.963457
38382,"(oil, whole milk)",(mustard),9.010536
